<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/notebook%2Faman/notebooks/Observablity_And_Tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

In [ ]:
pip install langchain-openai==0.2.3 langchain==0.3.4 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/5

## Setting up env

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY"
os.environ["LANGCHAIN_API_KEY"] = "<LANGCHAIN_API_KEY>"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "<LANGCHAIN_PROJECT>"

## Tracing Langchain calls

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub

prompt = hub.pull("wikianes/section_writer")
llm = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | llm
output = chain.invoke({"section_description": "Quantum Physics"})
print(output.content)

# Quantum Physics in Anesthesia

## Introduction to Quantum Physics in Medicine
Quantum physics, the study of matter and energy at the atomic and subatomic levels, has increasingly influenced various fields of medicine, including anesthesiology. Understanding the principles of quantum mechanics can enhance anesthesiologists' comprehension of drug interactions and the mechanisms of action of anesthetic agents, ultimately improving patient outcomes during surgical procedures.

## Quantum Effects on Drug Mechanisms
Anesthetic agents operate through complex interactions at the molecular level, influenced by quantum phenomena. For example, the binding affinity of anesthetic drugs to their target receptors can be affected by quantum tunneling and superposition. These principles can help explain variations in anesthesia depth and duration, as well as individual patient responses to anesthetic agents. Anesthesiologists should consider these quantum effects when selecting agents, especially in 

## Tracing OpenAI calls

In [ ]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

# wrap_openai from langsmith provides wrapper to
ai_client = wrap_openai(OpenAI())


def retrieve_documents(inquiry: str):
    docs = [
        "Vector databases enable efficient semantic search for LLMs.",
        "Retrieval-Augmented Generation (RAG) improves LLM accuracy with relevant context.",
        "Large Language Models (LLMs) have set new benchmarks in NLP tasks.",
        "Transformer architectures have revolutionized many areas of machine learning.",
        "Embedding models convert text into vector representations for similarity comparisons."
    ]
    return docs

def ask_qa(query):
    context = retrieve_documents(query)
    system_prompt = f"""
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    Context: {context}
    """

    return ai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
        model="gpt-4o-mini",
    )

In [ ]:
ask_qa("what is vector db?")

ChatCompletion(id='chatcmpl-ALZOOE9lCSqi9VO0o847Qwo3Xc2FI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A vector database is a type of database designed to store and manage vector representations of data, particularly for efficient semantic search and similarity comparisons. These vectors are often generated by embedding models, which convert text or other data types into numerical vectors. Vector databases enable quick retrieval of information by allowing for searches based on semantic similarity rather than relying solely on keyword matching. This functionality is particularly useful for applications involving Large Language Models (LLMs), as it facilitates Retrieval-Augmented Generation (RAG) by providing relevant context that enhances model accuracy.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729704300, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=N

## Using Langsmith traceable decorator

In [ ]:
from openai import OpenAI
from langsmith import traceable

openai_client = OpenAI()

@traceable(run_type="llm", name="OpenAI call")
def call_llm(model, messages):
    result = openai_client.chat.completions.create(
            messages=messages,
            model=model
        )
    return result.choices[0].message.content


@traceable(run_type="retriever")
def retrieve_documents(inquiry: str):
    docs = [
        "Vector databases enable efficient semantic search for LLMs.",
        "Retrieval-Augmented Generation (RAG) improves LLM accuracy with relevant context.",
        "Large Language Models (LLMs) have set new benchmarks in NLP tasks.",
        "Transformer architectures have revolutionized many areas of machine learning.",
        "Embedding models convert text into vector representations for similarity comparisons."
    ]
    return docs

@traceable(run_type="chain")
def ask_qa(query):
    context = retrieve_documents(query)
    system_prompt = f"""
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    Context: {context}
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    return call_llm("gpt-4o-mini", messages)

In [ ]:
ask_qa("What are large language models", langsmith_extra={"metadata": {"user": "test_user@gmail.com"}})

'Large Language Models (LLMs) are advanced AI systems designed to understand and generate human-like text. They have set new benchmarks in various natural language processing (NLP) tasks by utilizing massive datasets and sophisticated architectures, particularly transformer architectures. LLMs leverage techniques such as Retrieval-Augmented Generation (RAG) to enhance their accuracy and effectiveness by incorporating relevant context into their responses. Additionally, embedding models are often used in conjunction with LLMs to convert text into vector representations, facilitating efficient semantic search and similarity comparisons.'